In [1]:
import random
import os
import numpy as np
from datetime import datetime, timedelta

class ImprovedRealisticPortDataGenerator:
    """改进的现实化港口调度问题数据生成器 - 支持马力约束的多拖船组合服务"""
    
    def __init__(self, seed=42):
        """初始化数据生成器"""
        random.seed(seed)
        np.random.seed(seed)
        
        # 现实港口参数配置 - 平衡复杂度与可解性
        self.ship_type_config = {
            'container': {'size_range': (3, 5), 'ratio': 0.35, 'priority_range': (2.2, 3.0)},
            'bulk': {'size_range': (3, 5), 'ratio': 0.3, 'priority_range': (1.8, 2.5)},
            'tanker': {'size_range': (2, 4), 'ratio': 0.25, 'priority_range': (2.0, 2.8)},
            'general': {'size_range': (1, 3), 'ratio': 0.1, 'priority_range': (1.2, 2.0)}
        }
    
    def _get_time_period_ranges(self, time_periods):
        """根据T值获取各种时间参数的周期数范围"""
        if time_periods == 12:  # 每周期2小时
            duration_range = (1, 2)
            service_range = (1, 1)
            window_range = (1, 2)
            prep_time = 1
            tolerance = 1
        elif time_periods == 24:  # 每周期1小时
            duration_range = (1, 3)
            service_range = (1, 2)
            window_range = (1, 3)
            prep_time = 1
            tolerance = 2
        elif time_periods == 48:  # 每周期30分钟
            duration_range = (2, 6)
            service_range = (2, 4)
            window_range = (2, 6)
            prep_time = 2
            tolerance = 3
        else:
            duration_range = (1, 3)
            service_range = (1, 2)
            window_range = (1, 3)
            prep_time = 1
            tolerance = 2
        return duration_range, service_range, window_range, prep_time, tolerance
    
    def _calculate_balanced_costs(self, vessels, vessel_durations, tugboat_costs, time_periods):
        """计算平衡的多目标成本参数"""
        
        # 1. 估算各成本项的典型量级
        avg_duration = np.mean(vessel_durations)
        avg_tugboat_cost = np.mean(tugboat_costs)
        avg_priority = np.mean([v['priority'] for v in vessels])
        
        # 2. 估算单船各成本项的典型值
        # Z2: 港内时间成本 = 优先权重 × 等待成本单价 × (泊位时间 + 拖船服务时间)
        typical_port_time = avg_duration + 2  # 泊位时间 + 进出港时间
        typical_waiting_cost_per_period = 50  # 典型的单位时间等待成本
        typical_z2_per_vessel = avg_priority * typical_waiting_cost_per_period * typical_port_time
        
        # Z3: ETA偏差成本 = 优先权重 × JIT成本单价 × 典型偏差时间
        typical_deviation = 2  # 典型偏差2个时间周期
        typical_jit_cost_per_period = 20  # 典型的单位偏差成本
        typical_z3_per_vessel = avg_priority * typical_jit_cost_per_period * typical_deviation
        
        # Z4: 拖船成本 = 拖船单价 × 服务时间 (考虑多拖船组合)
        typical_service_time = 2  # 进港+出港服务时间
        typical_tugboat_count = 1.5  # 平均拖船使用数量
        typical_z4_per_vessel = avg_tugboat_cost * typical_service_time * typical_tugboat_count
        
        print(f"成本估算 - Z2典型值: {typical_z2_per_vessel:.1f}, Z3典型值: {typical_z3_per_vessel:.1f}, Z4典型值: {typical_z4_per_vessel:.1f}")
        
        # 3. 设计平衡的惩罚参数M
        # M应该是"不服务一艘船的机会成本"，即该船如果被服务的总成本的2-3倍
        single_vessel_service_cost = typical_z2_per_vessel + typical_z3_per_vessel + typical_z4_per_vessel
        penalty_parameter = single_vessel_service_cost * 2.5  # 2.5倍的服务成本作为惩罚
        
        # 4. 设计平衡的目标函数权重
        # 确保各目标项在优化中具有相当的影响力
        total_vessels = len(vessels)
        
        # 估算各目标项的总量级（假设所有船都被服务）
        total_z2_estimate = total_vessels * typical_z2_per_vessel
        total_z3_estimate = total_vessels * typical_z3_per_vessel  
        total_z4_estimate = total_vessels * typical_z4_per_vessel
        
        # Z1的期望值应该很小（理想情况下为0），但要考虑可能的未服务船舶
        estimated_unserved_ratio = 0.05  # 假设5%的船舶可能无法服务
        total_z1_estimate = penalty_parameter * avg_priority * total_vessels * estimated_unserved_ratio
        
        print(f"总量级估算 - Z1: {total_z1_estimate:.0f}, Z2: {total_z2_estimate:.0f}, Z3: {total_z3_estimate:.0f}, Z4: {total_z4_estimate:.0f}")
        
        # 计算权重，使得加权后的各项具有相似的量级影响
        base_scale = max(total_z2_estimate, total_z3_estimate, total_z4_estimate, total_z1_estimate)
        
        lambda_1 = base_scale / (total_z1_estimate + 1e-6)  # 避免除零
        lambda_2 = base_scale / (total_z2_estimate + 1e-6)
        lambda_3 = base_scale / (total_z3_estimate + 1e-6)
        lambda_4 = base_scale / (total_z4_estimate + 1e-6)
        
        # 归一化权重到合理范围[0.1, 1.0]
        total_weight = lambda_1 + lambda_2 + lambda_3 + lambda_4
        lambda_1 = (lambda_1 / total_weight) * 0.7 + 0.1  # Z1稍微高一点权重
        lambda_2 = (lambda_2 / total_weight) * 0.6 + 0.15  # Z2是主要成本
        lambda_3 = (lambda_3 / total_weight) * 0.4 + 0.1   # Z3适中权重
        lambda_4 = (lambda_4 / total_weight) * 0.3 + 0.05  # Z4相对较小权重
        
        objective_weights = [lambda_1, lambda_2, lambda_3, lambda_4]
        
        # 5. 根据平衡后的成本结构生成船舶成本参数
        vessel_waiting_costs = []
        vessel_jit_costs = []
        
        for vessel in vessels:
            # 等待成本：基于船舶大小和优先级
            if vessel['size'] >= 4:  # 大船
                base_waiting = random.uniform(40, 70)
            elif vessel['size'] >= 3:  # 中船
                base_waiting = random.uniform(25, 45)
            else:  # 小船
                base_waiting = random.uniform(15, 35)
            
            # 调整到合理水平，确保与其他成本项平衡
            waiting_cost = round(base_waiting * (typical_waiting_cost_per_period / 50), 1)
            vessel_waiting_costs.append(waiting_cost)
            
            # JIT偏差成本：通常为等待成本的30-50%
            jit_cost = round(waiting_cost * random.uniform(0.3, 0.5), 1)
            vessel_jit_costs.append(jit_cost)
        
        return penalty_parameter, objective_weights, vessel_waiting_costs, vessel_jit_costs
    
    def _generate_realistic_vessels(self, vessel_num, time_periods):
        """生成现实化的船舶数据"""
        vessels = []
        
        # 获取时间参数范围
        duration_range, service_range, window_range, _, _ = self._get_time_period_ranges(time_periods)
        
        # 按船型比例分配
        for ship_type, config in self.ship_type_config.items():
            count = int(vessel_num * config['ratio'])
            if ship_type == 'general':  
                count = vessel_num - len(vessels)
            
            for _ in range(count):
                vessel = {
                    'type': ship_type,
                    'size': random.randint(*config['size_range']),
                    'priority': round(random.uniform(*config['priority_range']), 1)
                }
                vessels.append(vessel)
        
        random.shuffle(vessels)
        
        # 生成到达时间
        vessel_etas = self._generate_realistic_etas(vessel_num, time_periods)
        
        # 泊位服务时间
        vessel_durations = []
        for vessel in vessels:
            if vessel['type'] == 'container':
                if vessel['size'] >= 4:
                    duration = random.randint(duration_range[1], duration_range[1])
                else:
                    duration = random.randint(duration_range[0], duration_range[1])
            elif vessel['type'] in ['tanker', 'bulk']:
                duration = random.randint(duration_range[0], duration_range[1])
            else:  # general
                duration = random.randint(duration_range[0], max(duration_range[0], duration_range[1]-1))
            vessel_durations.append(duration)
        
        # 进港服务时间
        vessel_inbound_times = []
        for vessel in vessels:
            if vessel['size'] >= 4:
                inbound_time = random.randint(service_range[1], service_range[1])
            else:
                inbound_time = random.randint(service_range[0], service_range[1])
            vessel_inbound_times.append(inbound_time)
        
        # 离港服务时间
        vessel_outbound_times = []
        for vessel in vessels:
            if vessel['size'] >= 4:
                outbound_time = random.randint(service_range[1], service_range[1])
            else:
                outbound_time = random.randint(service_range[0], service_range[1])
            vessel_outbound_times.append(outbound_time)
        
        # 马力需求 - 变体3新增
        vessel_horsepower_requirements = []
        for vessel in vessels:
            # 基于船舶大小和类型确定马力需求
            if vessel['size'] == 5:
                hp_requirement = random.randint(4500, 6000)
            elif vessel['size'] == 4:
                hp_requirement = random.randint(3000, 4500)
            elif vessel['size'] == 3:
                hp_requirement = random.randint(2000, 3000)
            elif vessel['size'] == 2:
                hp_requirement = random.randint(1200, 2000)
            else:  # size == 1
                hp_requirement = random.randint(800, 1200)
            
            # 特殊船型的马力调整
            if vessel['type'] == 'tanker':
                hp_requirement = int(hp_requirement * 1.1)  # 油轮需要更多马力
            elif vessel['type'] == 'container' and vessel['size'] >= 4:
                hp_requirement = int(hp_requirement * 1.05)  # 大型集装箱船
            
            vessel_horsepower_requirements.append(hp_requirement)
        
        # 时间窗
        vessel_early_limits = []
        for vessel in vessels:
            if vessel['type'] == 'container':
                early_limit = random.randint(window_range[0], max(window_range[0], window_range[1]-1))
            else:
                early_limit = random.randint(window_range[0], window_range[1])
            vessel_early_limits.append(early_limit)
        
        vessel_late_limits = []
        for vessel in vessels:
            if vessel['type'] == 'container':
                late_limit = random.randint(window_range[0], max(window_range[0], window_range[1]-1))
            else:
                late_limit = random.randint(window_range[0], window_range[1])
            vessel_late_limits.append(late_limit)
        
        vessel_sizes = [v['size'] for v in vessels]
        vessel_priorities = [v['priority'] for v in vessels]
        
        return (vessels, vessel_sizes, vessel_etas, vessel_durations, vessel_inbound_times,
                vessel_outbound_times, vessel_priorities, vessel_early_limits, vessel_late_limits,
                vessel_horsepower_requirements)
    
    def _generate_realistic_etas(self, vessel_num, time_periods):
        """生成现实化的ETA分布"""
        vessel_etas = []
        
        main_arrivals = int(vessel_num * 0.7)
        mid_arrivals = int(vessel_num * 0.3)
        late_arrivals = vessel_num - main_arrivals - mid_arrivals
        
        main_period = max(4, time_periods // 3)
        mid_start = main_period + 1
        mid_end = max(mid_start + 3, time_periods * 2 // 3)
        
        for _ in range(main_arrivals):
            eta = random.randint(1, main_period)
            vessel_etas.append(eta)
        
        for _ in range(mid_arrivals):
            eta = random.randint(mid_start, mid_end)
            vessel_etas.append(eta)
        
        for _ in range(late_arrivals):
            eta = random.randint(mid_end + 1, max(mid_end + 2, time_periods - 2))
            vessel_etas.append(eta)
        
        random.shuffle(vessel_etas)
        return vessel_etas
    
    def _generate_realistic_berths(self, berth_num, max_vessel_size):
        """生成现实化的泊位配置"""
        berth_capacities = []
        
        if berth_num >= 25:
            large_berths = max(2, berth_num * 3 // 10)
            berth_capacities.extend([5] * large_berths)
            medium_large_berths = berth_num * 4 // 10
            berth_capacities.extend([4] * medium_large_berths)
            medium_berths = berth_num // 5
            berth_capacities.extend([3] * medium_berths)
            small_berths = berth_num - len(berth_capacities)
            berth_capacities.extend([2] * small_berths)
        elif berth_num >= 15:
            large_berths = max(2, berth_num // 4)
            berth_capacities.extend([5] * large_berths)
            medium_large_berths = berth_num * 45 // 100
            berth_capacities.extend([4] * medium_large_berths)
            medium_berths = berth_num // 5
            berth_capacities.extend([3] * medium_berths)
            small_berths = berth_num - len(berth_capacities)
            berth_capacities.extend([2] * small_berths)
        else:
            large_berths = max(1, berth_num // 4)
            berth_capacities.extend([5] * large_berths)
            medium_large_berths = berth_num * 2 // 5
            berth_capacities.extend([4] * medium_large_berths)
            medium_berths = berth_num // 4
            berth_capacities.extend([3] * medium_berths)
            small_berths = berth_num - len(berth_capacities)
            berth_capacities.extend([2] * small_berths)
        
        # 确保大船有足够选择
        large_capacity_berths = sum(1 for cap in berth_capacities if cap >= 4)
        target_large_berths = max(berth_num // 3, berth_num // 2)
        
        if large_capacity_berths < target_large_berths:
            for i in range(len(berth_capacities)):
                if berth_capacities[i] == 3 and large_capacity_berths < target_large_berths:
                    berth_capacities[i] = 4
                    large_capacity_berths += 1
        
        return berth_capacities
    
    def _generate_realistic_tugboats(self, tugboat_num, vessel_horsepower_requirements):
        """生成现实化的拖船配置 - 基于马力系统"""
        tugboat_horsepower = []
        tugboat_costs = []
        
        # 分析船舶马力需求分布来设计拖船马力分布
        max_hp_requirement = max(vessel_horsepower_requirements) if vessel_horsepower_requirements else 3000
        avg_hp_requirement = np.mean(vessel_horsepower_requirements) if vessel_horsepower_requirements else 2000
        
        # 创建多样化的拖船马力分布
        # 大型拖船：3500-5000 HP，能处理最大船舶
        # 中大型拖船：2500-3500 HP，处理中大型船舶
        # 中型拖船：1500-2500 HP，处理中型船舶
        # 小型拖船：800-1500 HP，处理小型船舶
        
        if tugboat_num >= 60:
            # 大规模港口：多样化配置
            large_tugs = max(3, tugboat_num * 15 // 100)  # 15%大型拖船
            medium_large_tugs = tugboat_num * 35 // 100   # 35%中大型拖船
            medium_tugs = tugboat_num * 35 // 100         # 35%中型拖船
            small_tugs = tugboat_num - large_tugs - medium_large_tugs - medium_tugs  # 剩余小型拖船
        elif tugboat_num >= 30:
            # 中规模港口
            large_tugs = max(2, tugboat_num * 12 // 100)  # 12%大型拖船
            medium_large_tugs = tugboat_num * 38 // 100   # 38%中大型拖船
            medium_tugs = tugboat_num * 35 // 100         # 35%中型拖船
            small_tugs = tugboat_num - large_tugs - medium_large_tugs - medium_tugs
        else:
            # 小规模港口
            large_tugs = max(1, tugboat_num * 10 // 100)  # 10%大型拖船
            medium_large_tugs = tugboat_num * 40 // 100   # 40%中大型拖船
            medium_tugs = tugboat_num * 35 // 100         # 35%中型拖船
            small_tugs = tugboat_num - large_tugs - medium_large_tugs - medium_tugs
        
        # 生成各类拖船
        # 大型拖船：3500-5000 HP
        for _ in range(large_tugs):
            hp = random.randint(3500, 5000)
            tugboat_horsepower.append(hp)
            # 大型拖船成本较高
            cost = round(random.uniform(80, 120), 1)
            tugboat_costs.append(cost)
        
        # 中大型拖船：2500-3500 HP
        for _ in range(medium_large_tugs):
            hp = random.randint(2500, 3500)
            tugboat_horsepower.append(hp)
            cost = round(random.uniform(60, 90), 1)
            tugboat_costs.append(cost)
        
        # 中型拖船：1500-2500 HP
        for _ in range(medium_tugs):
            hp = random.randint(1500, 2500)
            tugboat_horsepower.append(hp)
            cost = round(random.uniform(40, 65), 1)
            tugboat_costs.append(cost)
        
        # 小型拖船：800-1500 HP
        for _ in range(small_tugs):
            hp = random.randint(800, 1500)
            tugboat_horsepower.append(hp)
            cost = round(random.uniform(25, 45), 1)
            tugboat_costs.append(cost)
        
        # 打乱顺序
        combined = list(zip(tugboat_horsepower, tugboat_costs))
        random.shuffle(combined)
        tugboat_horsepower, tugboat_costs = zip(*combined)
        tugboat_horsepower = list(tugboat_horsepower)
        tugboat_costs = list(tugboat_costs)
        
        # 智能调整：确保高马力需求的船舶有足够的拖船可用
        if max_hp_requirement > 4000:
            high_power_tugs = sum(1 for hp in tugboat_horsepower if hp >= 4000)
            high_requirement_vessels = sum(1 for req in vessel_horsepower_requirements if req >= 4000)
            
            if high_power_tugs < max(2, high_requirement_vessels // 2):
                # 升级一些拖船到高马力
                upgrade_count = max(2, high_requirement_vessels // 2) - high_power_tugs
                upgrade_candidates = []
                
                for i, hp in enumerate(tugboat_horsepower):
                    if 3000 <= hp < 4000 and len(upgrade_candidates) < upgrade_count:
                        upgrade_candidates.append(i)
                
                for idx in upgrade_candidates:
                    tugboat_horsepower[idx] = random.randint(4000, 5000)
                    tugboat_costs[idx] = round(random.uniform(90, 130), 1)
        
        return tugboat_horsepower, tugboat_costs
    
    def _get_time_unit_info(self, time_periods):
        """获取时间单位信息"""
        if time_periods == 12:
            return "2 hours", 2.0
        elif time_periods == 24:
            return "1 hour", 1.0
        elif time_periods == 48:
            return "30 minutes", 0.5
        else:
            return "1 hour", 1.0
    
    def _print_balanced_cost_analysis(self, penalty_parameter, objective_weights, 
                                    vessel_waiting_costs, vessel_jit_costs, tugboat_costs,
                                    vessels, vessel_durations):
        """打印平衡成本分析"""
        print(f"\n=== 平衡多目标成本分析 ===")
        
        # 惩罚参数分析
        print(f"惩罚参数 M: {penalty_parameter:.1f}")
        
        # 目标权重分析
        print(f"目标权重 [λ₁, λ₂, λ₃, λ₄]: {[round(w, 3) for w in objective_weights]}")
        
        # 成本范围分析
        print(f"船舶等待成本范围: {min(vessel_waiting_costs):.1f} - {max(vessel_waiting_costs):.1f}")
        print(f"船舶JIT偏差成本范围: {min(vessel_jit_costs):.1f} - {max(vessel_jit_costs):.1f}")
        print(f"拖船成本范围: {min(tugboat_costs):.1f} - {max(tugboat_costs):.1f}")
        
        # 典型成本估算
        avg_priority = np.mean([v['priority'] for v in vessels])
        avg_duration = np.mean(vessel_durations)
        avg_waiting_cost = np.mean(vessel_waiting_costs)
        avg_jit_cost = np.mean(vessel_jit_costs)
        avg_tugboat_cost = np.mean(tugboat_costs)
        
        typical_z1 = penalty_parameter * avg_priority
        typical_z2 = avg_priority * avg_waiting_cost * (avg_duration + 2)
        typical_z3 = avg_priority * avg_jit_cost * 2
        typical_z4 = avg_tugboat_cost * 2 * 1.5  # 考虑多拖船组合
        
        print(f"单船典型成本估算:")
        print(f"  Z₁ (不服务惩罚): {typical_z1:.1f}")
        print(f"  Z₂ (港内时间): {typical_z2:.1f}")
        print(f"  Z₃ (ETA偏差): {typical_z3:.1f}")
        print(f"  Z₄ (拖船使用): {typical_z4:.1f}")
        
        # 加权后影响分析
        weighted_z1 = typical_z1 * objective_weights[0]
        weighted_z2 = typical_z2 * objective_weights[1]
        weighted_z3 = typical_z3 * objective_weights[2]
        weighted_z4 = typical_z4 * objective_weights[3]
        
        print(f"加权后单船成本影响:")
        print(f"  λ₁Z₁: {weighted_z1:.1f}")
        print(f"  λ₂Z₂: {weighted_z2:.1f}")
        print(f"  λ₃Z₃: {weighted_z3:.1f}")
        print(f"  λ₄Z₄: {weighted_z4:.1f}")
        
        total_weighted = weighted_z1 + weighted_z2 + weighted_z3 + weighted_z4
        print(f"加权总成本: {total_weighted:.1f}")
        print(f"各项占比: Z₁:{weighted_z1/total_weighted:.1%}, Z₂:{weighted_z2/total_weighted:.1%}, Z₃:{weighted_z3/total_weighted:.1%}, Z₄:{weighted_z4/total_weighted:.1%}")
    
    def generate_balanced_case(self, vessel_num, berth_num, tugboat_num, time_periods=24, 
                              case_name=None, output_dir="balanced_cases"):
        """生成平衡的港口调度用例 - 支持马力约束系统"""
        
        # 自动生成简洁命名
        if case_name is None:
            case_name = f"{vessel_num}_{berth_num}_{tugboat_num}_T{time_periods}"
        
        os.makedirs(output_dir, exist_ok=True)
        
        if time_periods not in [12, 24, 48]:
            print(f"Warning: time_periods {time_periods} not in [12, 24, 48], using 24")
            time_periods = 24
        
        # 生成船舶数据
        (vessels, vessel_sizes, vessel_etas, vessel_durations, vessel_inbound_times,
         vessel_outbound_times, vessel_priorities, vessel_early_limits, 
         vessel_late_limits, vessel_horsepower_requirements) = self._generate_realistic_vessels(vessel_num, time_periods)
        
        # 生成泊位配置
        berth_capacities = self._generate_realistic_berths(berth_num, max(vessel_sizes))
        
        # 生成拖船配置 - 基于马力系统
        tugboat_horsepower, tugboat_costs = self._generate_realistic_tugboats(tugboat_num, vessel_horsepower_requirements)
        
        # 计算平衡的成本参数
        penalty_parameter, objective_weights, vessel_waiting_costs, vessel_jit_costs = \
            self._calculate_balanced_costs(vessels, vessel_durations, tugboat_costs, time_periods)
        
        # 系统参数
        _, _, _, prep_time, tolerance = self._get_time_period_ranges(time_periods)
        inbound_preparation_time = prep_time
        outbound_preparation_time = prep_time
        time_constraint_tolerance = tolerance
        
        # 拖船数量限制 - 变体3新增
        if tugboat_num >= 50:
            max_tugboats_per_service = 3
        elif tugboat_num >= 20:
            max_tugboats_per_service = 2
        else:
            max_tugboats_per_service = 2
        
        # 写入文件
        filename = os.path.join(output_dir, f"{case_name}.txt")
        time_unit, hours_per_period = self._get_time_unit_info(time_periods)
        
        with open(filename, 'w') as f:
            f.write("# Balanced Multi-Objective Port Scheduling Problem Instance with Horsepower Constraints\n")
            f.write("# Mathematical model with multi-tugboat combinations and horsepower requirements\n")
            f.write(f"# Time configuration: T={time_periods}, each period = {time_unit}\n")
            f.write("# Format: parameter_name = value or [array values]\n\n")
            
            # 基本维度
            f.write("# Basic dimensions\n")
            f.write(f"vessel_num = {vessel_num}\n")
            f.write(f"berth_num = {berth_num}\n")
            f.write(f"tugboat_num = {tugboat_num}\n")
            f.write(f"time_periods = {time_periods}\n\n")
            
            # 船舶数据
            f.write(f"# Vessel data ({vessel_num} vessels) - All times in INTEGER periods\n")
            f.write(f"vessel_sizes = {vessel_sizes}\n")
            f.write(f"vessel_etas = {vessel_etas}\n")
            f.write(f"vessel_durations = {vessel_durations}\n")
            f.write(f"vessel_inbound_service_times = {vessel_inbound_times}\n")
            f.write(f"vessel_outbound_service_times = {vessel_outbound_times}\n")
            f.write(f"vessel_priority_weights = {vessel_priorities}\n")
            f.write(f"vessel_waiting_costs = {vessel_waiting_costs}\n")
            f.write(f"vessel_jit_costs = {vessel_jit_costs}\n")
            f.write(f"vessel_horsepower_requirements = {vessel_horsepower_requirements}\n")
            f.write(f"vessel_early_limits = {vessel_early_limits}\n")
            f.write(f"vessel_late_limits = {vessel_late_limits}\n\n")
            
            # 泊位数据
            f.write(f"# Berth data ({berth_num} berths)\n")
            f.write(f"berth_capacities = {berth_capacities}\n\n")
            
            # 拖船数据 - 马力系统
            f.write(f"# Tugboat data ({tugboat_num} tugboats) - Horsepower-based system\n")
            f.write(f"tugboat_horsepower = {tugboat_horsepower}\n")
            f.write(f"tugboat_costs = {tugboat_costs}\n\n")
            
            # 系统参数
            f.write("# System parameters for multi-tugboat combinations\n")
            f.write(f"inbound_preparation_time = {inbound_preparation_time}\n")
            f.write(f"outbound_preparation_time = {outbound_preparation_time}\n")
            f.write(f"max_tugboats_per_service = {max_tugboats_per_service}\n")
            f.write(f"time_constraint_tolerance = {time_constraint_tolerance}\n")
            f.write(f"penalty_parameter = {penalty_parameter}\n")
            f.write(f"objective_weights = {objective_weights}\n\n")
            
            # 数学模型参数
            f.write("# Mathematical model parameters (horsepower-based)\n")
            f.write(f"M = {penalty_parameter}\n")
            f.write(f"lambda_1 = {objective_weights[0]}\n")
            f.write(f"lambda_2 = {objective_weights[1]}\n")
            f.write(f"lambda_3 = {objective_weights[2]}\n")
            f.write(f"lambda_4 = {objective_weights[3]}\n")
            f.write(f"epsilon_time = {time_constraint_tolerance}\n")
            f.write(f"rho_in = {inbound_preparation_time}\n")
            f.write(f"rho_out = {outbound_preparation_time}\n")
            f.write(f"H_max = {max_tugboats_per_service}\n")
            f.write(f"tau_in = {vessel_inbound_times}\n")
            f.write(f"tau_out = {vessel_outbound_times}\n")
            f.write(f"alpha = {vessel_priorities}\n")
            f.write(f"beta = {vessel_waiting_costs}\n")
            f.write(f"gamma = {vessel_jit_costs}\n")
            f.write(f"P_k = {tugboat_horsepower}\n")
            f.write(f"P_req = {vessel_horsepower_requirements}\n")
            f.write(f"c_k = {tugboat_costs}\n")
            f.write(f"Delta_early = {vessel_early_limits}\n")
            f.write(f"Delta_late = {vessel_late_limits}\n")
        
        # 打印马力约束分析
        self._print_horsepower_analysis(vessel_horsepower_requirements, tugboat_horsepower, max_tugboats_per_service)
        
        # 打印平衡成本分析
        self._print_balanced_cost_analysis(penalty_parameter, objective_weights, 
                                         vessel_waiting_costs, vessel_jit_costs, tugboat_costs,
                                         vessels, vessel_durations)
        
        # 打印基础分析
        self._print_realistic_analysis(vessel_sizes, berth_capacities, tugboat_horsepower, 
                                     vessel_num, berth_num, tugboat_num, time_periods,
                                     vessel_durations, vessel_inbound_times, vessel_outbound_times,
                                     vessel_horsepower_requirements)
        
        print(f"\n生成平衡多目标用例: {filename}")
        return filename
    
    def _print_horsepower_analysis(self, vessel_horsepower_requirements, tugboat_horsepower, max_tugboats_per_service):
        """打印马力约束分析"""
        print(f"\n=== 马力约束系统分析 ===")
        
        # 船舶马力需求分析
        min_req = min(vessel_horsepower_requirements)
        max_req = max(vessel_horsepower_requirements)
        avg_req = np.mean(vessel_horsepower_requirements)
        print(f"船舶马力需求范围: {min_req} - {max_req} HP (平均: {avg_req:.0f} HP)")
        
        # 拖船马力供给分析
        min_supply = min(tugboat_horsepower)
        max_supply = max(tugboat_horsepower)
        avg_supply = np.mean(tugboat_horsepower)
        print(f"拖船马力供给范围: {min_supply} - {max_supply} HP (平均: {avg_supply:.0f} HP)")
        
        # 马力匹配性分析
        high_requirement_vessels = sum(1 for req in vessel_horsepower_requirements if req >= 4000)
        high_power_tugboats = sum(1 for hp in tugboat_horsepower if hp >= 4000)
        medium_requirement_vessels = sum(1 for req in vessel_horsepower_requirements if 2000 <= req < 4000)
        medium_power_tugboats = sum(1 for hp in tugboat_horsepower if 2000 <= hp < 4000)
        
        print(f"高马力需求船舶(≥4000HP): {high_requirement_vessels}艘")
        print(f"高马力拖船(≥4000HP): {high_power_tugboats}艘")
        print(f"中等马力需求船舶(2000-4000HP): {medium_requirement_vessels}艘")
        print(f"中等马力拖船(2000-4000HP): {medium_power_tugboats}艘")
        
        # 组合服务能力分析
        print(f"最大拖船组合数量: {max_tugboats_per_service}艘/服务")
        max_combined_power = sorted(tugboat_horsepower, reverse=True)[:max_tugboats_per_service]
        max_achievable = sum(max_combined_power)
        print(f"最大可组合马力: {max_achievable} HP")
        
        # 可解性检查
        unsatisfiable_vessels = sum(1 for req in vessel_horsepower_requirements if req > max_achievable)
        if unsatisfiable_vessels > 0:
            print(f"⚠️ 警告: {unsatisfiable_vessels}艘船舶的马力需求超过最大可组合马力")
        else:
            print(f"✓ 所有船舶的马力需求都可以通过拖船组合满足")
    
    def _print_realistic_analysis(self, vessel_sizes, berth_capacities, tugboat_horsepower, 
                                vessel_num, berth_num, tugboat_num, time_periods, 
                                vessel_durations, vessel_inbound_times, vessel_outbound_times,
                                vessel_horsepower_requirements):
        """打印现实化分析"""
        time_unit, hours_per_period = self._get_time_unit_info(time_periods)
        
        print(f"\n=== 现实化港口调度案例分析 (马力约束系统) ===")
        print(f"时间配置: T={time_periods}, 每周期={time_unit}")
        print(f"船舶数量: {vessel_num}, 泊位数量: {berth_num}, 拖船数量: {tugboat_num}")
        
        # 船舶规模分析
        size_dist = {}
        for size in vessel_sizes:
            size_dist[size] = size_dist.get(size, 0) + 1
        print(f"船舶规模分布: {dict(sorted(size_dist.items()))}")
        
        # 泊位容量分析
        berth_dist = {}
        for cap in berth_capacities:
            berth_dist[cap] = berth_dist.get(cap, 0) + 1
        print(f"泊位容量分布: {dict(sorted(berth_dist.items()))}")
        
        # 拖船马力分析
        hp_ranges = {
            "800-1500": sum(1 for hp in tugboat_horsepower if 800 <= hp < 1500),
            "1500-2500": sum(1 for hp in tugboat_horsepower if 1500 <= hp < 2500),
            "2500-3500": sum(1 for hp in tugboat_horsepower if 2500 <= hp < 3500),
            "3500-5000": sum(1 for hp in tugboat_horsepower if 3500 <= hp <= 5000)
        }
        print(f"拖船马力分布: {hp_ranges}")
        
        # 时间分析
        avg_duration = sum(vessel_durations) / len(vessel_durations)
        avg_inbound = sum(vessel_inbound_times) / len(vessel_inbound_times)
        avg_outbound = sum(vessel_outbound_times) / len(vessel_outbound_times)
        
        print(f"平均泊位服务时间: {avg_duration:.1f}周期 ({avg_duration*hours_per_period:.1f}小时)")
        print(f"平均进港服务时间: {avg_inbound:.1f}周期 ({avg_inbound*hours_per_period:.1f}小时)")
        print(f"平均离港服务时间: {avg_outbound:.1f}周期 ({avg_outbound*hours_per_period:.1f}小时)")
        
        # 容量匹配分析
        large_ships = sum(1 for size in vessel_sizes if size >= 4)
        large_berths = sum(1 for cap in berth_capacities if cap >= 4)
        high_power_tugs = sum(1 for hp in tugboat_horsepower if hp >= 3000)
        
        print(f"大船(≥4级): {large_ships}艘")
        print(f"大泊位(≥4级): {large_berths}个")
        print(f"高马力拖船(≥3000HP): {high_power_tugs}艘")
        
        if large_ships > 0:
            berth_ratio = large_berths / large_ships
            tug_ratio = high_power_tugs / large_ships
            print(f"大船资源配比 - 泊位: {berth_ratio:.2f}, 高马力拖船: {tug_ratio:.2f}")

# 使用示例和测试
if __name__ == "__main__":
    generator = ImprovedRealisticPortDataGenerator(seed=42)
    
    # 生成20个小规模测试用例 (2-10艘船)
    small_scale_cases = [
        # 2-3艘船 - 超小规模
        (4, 2, 3, 12),
        (5, 3, 4, 24),
        (5, 2, 4, 12),
        (6, 3, 5, 24),
        
        # 4-5艘船 - 微小规模
        (9, 3, 5, 12),
        (7, 3, 6, 24),
        (6, 3, 6, 12),
        (7, 4, 7, 24),
        
        # 6-7艘船 - 小规模
        (9, 4, 8, 12),
        (10, 4, 8, 24),
        (11, 4, 9, 12),
        (13, 5, 10, 24),
        
        # 8-10艘船 - 中小规模
        (14, 5, 10, 12),
        (16, 5, 12, 24),
        (18, 5, 12, 12),
        (19, 6, 14, 24),
        (20, 5, 12, 12),
        (20, 6, 15, 24),
        (30, 6, 14, 48),
        (30, 7, 16, 48),
    ]
    
    # 生成平衡的测试用例 - 简洁命名: 船数_泊位数_拖船数_T值
    balanced_cases = [
        # 小规模测试用例
        (100, 15, 30, 24),
        (100, 15, 30, 48),
        
        # 中等规模测试用例
        (50, 7, 20, 24),
        (50, 7, 20, 48),
        
        # 大规模测试用例
        (20, 5, 15, 24),
        (20, 5, 15, 48),
        
        # 超大规模测试用例
        (30, 5, 15, 24),
        (30, 5, 15, 48),
         # 超大规模测试用例
        (70, 7, 15, 24),
        (70, 7, 15, 48),
        
        (80, 7, 20, 24),
        (80, 7, 20, 48),
        
        (90, 8, 20, 24),
        (90, 8, 20, 48),
        
        (150, 20, 40, 24),
        (150, 20, 40, 48),
        
        (100, 18, 40, 24),
        (100, 18, 40, 48),
    ]
    
    print("=== 生成20个小规模测试用例 (2-10艘船) ===")
    for vessel_num, berth_num, tugboat_num, time_periods in small_scale_cases:
        # 简洁命名格式: 船数_泊位数_拖船数_T值
        case_name = f"{vessel_num}_{berth_num}_{tugboat_num}_T{time_periods}"
        
        print(f"\n正在生成: {case_name}")
        print(f"  规模: {vessel_num}艘船, {berth_num}个泊位, {tugboat_num}艘拖船, T={time_periods}")
        
        generator.generate_balanced_case(
            vessel_num=vessel_num,
            berth_num=berth_num, 
            tugboat_num=tugboat_num,
            time_periods=time_periods,
            case_name=case_name,
            output_dir="small_cases_horsepower"  # 区分马力约束版本
        )
    
    print(f"\n{'='*60}")
    print("=== 生成平衡多目标测试用例 (马力约束系统) ===")
    for vessel_num, berth_num, tugboat_num, time_periods in balanced_cases:
        # 简洁命名格式: 船数_泊位数_拖船数_T值
        case_name = f"{vessel_num}_{berth_num}_{tugboat_num}_T{time_periods}"
        
        print(f"\n正在生成: {case_name}")
        print(f"  规模: {vessel_num}艘船, {berth_num}个泊位, {tugboat_num}艘拖船, T={time_periods}")
        
        generator.generate_balanced_case(
            vessel_num=vessel_num,
            berth_num=berth_num, 
            tugboat_num=tugboat_num,
            time_periods=time_periods,
            case_name=case_name,
            output_dir="balanced_cases_horsepower"  # 区分马力约束版本
        )
        
    print(f"\n{'='*60}")
    print("=== 所有测试用例生成完毕 (马力约束系统) ===")
    print("小规模用例(2-10艘船)保存在 ./small_cases_horsepower/ 目录下")
    print("平衡多目标用例保存在 ./balanced_cases_horsepower/ 目录下")
    print("\n马力约束系统特点:")
    print("- 船舶有具体的马力需求(800-6000 HP)")
    print("- 拖船有具体的马力供给(800-5000 HP)")
    print("- 支持多拖船组合服务(最多2-3艘)")
    print("- 马力约束确保组合拖船总马力≥船舶需求")
    print("- 智能匹配高马力需求船舶与高马力拖船")
    print("\n主要改进:")
    print("1. 从容量级别系统改为实际马力值系统")
    print("2. 添加vessel_horsepower_requirements参数")
    print("3. 修改为tugboat_horsepower参数(替代tugboat_capacities)")
    print("4. 添加max_tugboats_per_service参数")
    print("5. 智能拖船配置确保高需求船舶的可服务性")
    print("6. 提供详细的马力匹配性分析")

=== 生成20个小规模测试用例 (2-10艘船) ===

正在生成: 4_2_3_T12
  规模: 4艘船, 2个泊位, 3艘拖船, T=12
成本估算 - Z2典型值: 354.4, Z3典型值: 81.0, Z4典型值: 230.2
总量级估算 - Z1: 674, Z2: 1418, Z3: 324, Z4: 921

=== 马力约束系统分析 ===
船舶马力需求范围: 1190 - 5189 HP (平均: 3184 HP)
拖船马力供给范围: 2133 - 4275 HP (平均: 3069 HP)
高马力需求船舶(≥4000HP): 2艘
高马力拖船(≥4000HP): 1艘
中等马力需求船舶(2000-4000HP): 0艘
中等马力拖船(2000-4000HP): 2艘
最大拖船组合数量: 2艘/服务
最大可组合马力: 7075 HP
✓ 所有船舶的马力需求都可以通过拖船组合满足

=== 平衡多目标成本分析 ===
惩罚参数 M: 1663.9
目标权重 [λ₁, λ₂, λ₃, λ₄]: [0.263, 0.217, 0.294, 0.101]
船舶等待成本范围: 16.6 - 46.8
船舶JIT偏差成本范围: 5.8 - 16.7
拖船成本范围: 62.1 - 84.9
单船典型成本估算:
  Z₁ (不服务惩罚): 3369.5
  Z₂ (港内时间): 240.1
  Z₃ (ETA偏差): 47.4
  Z₄ (拖船使用): 230.2
加权后单船成本影响:
  λ₁Z₁: 887.1
  λ₂Z₂: 52.0
  λ₃Z₃: 13.9
  λ₄Z₄: 23.3
加权总成本: 976.3
各项占比: Z₁:90.9%, Z₂:5.3%, Z₃:1.4%, Z₄:2.4%

=== 现实化港口调度案例分析 (马力约束系统) ===
时间配置: T=12, 每周期=2 hours
船舶数量: 4, 泊位数量: 2, 拖船数量: 3
船舶规模分布: {1: 1, 2: 1, 5: 2}
泊位容量分布: {2: 1, 5: 1}
拖船马力分布: {'800-1500': 0, '1500-2500': 1, '2500-3500': 1, '3500-5000': 1}
平均泊位服务时间: 1.5周期 (3.0小时)
平均进港服务时间:

In [1]:
import random
import os
import numpy as np
from datetime import datetime, timedelta

class ImprovedRealisticPortDataGenerator:
    """改进的现实化港口调度问题数据生成器 - 支持马力约束的多拖船组合服务"""
    
    def __init__(self, seed=42):
        """初始化数据生成器"""
        random.seed(seed)
        np.random.seed(seed)
        
        # 现实港口参数配置 - 平衡复杂度与可解性
        self.ship_type_config = {
            'container': {'size_range': (3, 5), 'ratio': 0.35, 'priority_range': (2.2, 3.0)},
            'bulk': {'size_range': (3, 5), 'ratio': 0.3, 'priority_range': (1.8, 2.5)},
            'tanker': {'size_range': (2, 4), 'ratio': 0.25, 'priority_range': (2.0, 2.8)},
            'general': {'size_range': (1, 3), 'ratio': 0.1, 'priority_range': (1.2, 2.0)}
        }
    
    def _get_time_period_ranges(self, time_periods):
        """根据T值获取各种时间参数的周期数范围"""
        if time_periods == 12:  # 每周期2小时
            duration_range = (1, 2)
            service_range = (1, 1)
            window_range = (1, 2)
            prep_time = 1
            tolerance = 1
        elif time_periods == 24:  # 每周期1小时
            duration_range = (1, 3)
            service_range = (1, 2)
            window_range = (1, 3)
            prep_time = 1
            tolerance = 2
        elif time_periods == 48:  # 每周期30分钟
            duration_range = (2, 6)
            service_range = (2, 4)
            window_range = (2, 6)
            prep_time = 2
            tolerance = 3
        else:
            duration_range = (1, 3)
            service_range = (1, 2)
            window_range = (1, 3)
            prep_time = 1
            tolerance = 2
        return duration_range, service_range, window_range, prep_time, tolerance
    
    def _calculate_balanced_costs(self, vessels, vessel_durations, tugboat_costs, time_periods):
        """计算平衡的多目标成本参数"""
        
        # 估算各成本项的典型量级
        avg_duration = np.mean(vessel_durations)
        avg_tugboat_cost = np.mean(tugboat_costs)
        avg_priority = np.mean([v['priority'] for v in vessels])
        
        # 估算单船各成本项的典型值
        typical_port_time = avg_duration + 2
        typical_waiting_cost_per_period = 50
        typical_z2_per_vessel = avg_priority * typical_waiting_cost_per_period * typical_port_time
        
        typical_deviation = 2
        typical_jit_cost_per_period = 20
        typical_z3_per_vessel = avg_priority * typical_jit_cost_per_period * typical_deviation
        
        # 多拖船服务 - 典型用2艘拖船
        typical_tugboats_per_service = 2
        typical_service_time = 2
        typical_z4_per_vessel = avg_tugboat_cost * typical_tugboats_per_service * typical_service_time
        
        print(f"成本估算 - Z2典型值: {typical_z2_per_vessel:.1f}, Z3典型值: {typical_z3_per_vessel:.1f}, Z4典型值: {typical_z4_per_vessel:.1f}")
        
        # 设计平衡的惩罚参数M
        single_vessel_service_cost = typical_z2_per_vessel + typical_z3_per_vessel + typical_z4_per_vessel
        penalty_parameter = single_vessel_service_cost * 2.5
        
        # 设计平衡的目标函数权重
        total_vessels = len(vessels)
        
        total_z2_estimate = total_vessels * typical_z2_per_vessel
        total_z3_estimate = total_vessels * typical_z3_per_vessel  
        total_z4_estimate = total_vessels * typical_z4_per_vessel
        
        estimated_unserved_ratio = 0.05
        total_z1_estimate = penalty_parameter * avg_priority * total_vessels * estimated_unserved_ratio
        
        print(f"总量级估算 - Z1: {total_z1_estimate:.0f}, Z2: {total_z2_estimate:.0f}, Z3: {total_z3_estimate:.0f}, Z4: {total_z4_estimate:.0f}")
        
        # 计算权重
        base_scale = max(total_z2_estimate, total_z3_estimate, total_z4_estimate, total_z1_estimate)
        
        lambda_1 = base_scale / (total_z1_estimate + 1e-6)
        lambda_2 = base_scale / (total_z2_estimate + 1e-6)
        lambda_3 = base_scale / (total_z3_estimate + 1e-6)
        lambda_4 = base_scale / (total_z4_estimate + 1e-6)
        
        # 归一化权重
        total_weight = lambda_1 + lambda_2 + lambda_3 + lambda_4
        lambda_1 = (lambda_1 / total_weight) * 0.7 + 0.1
        lambda_2 = (lambda_2 / total_weight) * 0.6 + 0.15
        lambda_3 = (lambda_3 / total_weight) * 0.4 + 0.1
        lambda_4 = (lambda_4 / total_weight) * 0.3 + 0.05
        
        objective_weights = [lambda_1, lambda_2, lambda_3, lambda_4]
        
        # 生成船舶成本参数
        vessel_waiting_costs = []
        vessel_jit_costs = []
        
        for vessel in vessels:
            if vessel['size'] >= 4:
                base_waiting = random.uniform(40, 70)
            elif vessel['size'] >= 3:
                base_waiting = random.uniform(25, 45)
            else:
                base_waiting = random.uniform(15, 35)
            
            waiting_cost = round(base_waiting * (typical_waiting_cost_per_period / 50), 1)
            vessel_waiting_costs.append(waiting_cost)
            
            jit_cost = round(waiting_cost * random.uniform(0.3, 0.5), 1)
            vessel_jit_costs.append(jit_cost)
        
        return penalty_parameter, objective_weights, vessel_waiting_costs, vessel_jit_costs
    
    def _generate_realistic_vessels(self, vessel_num, time_periods):
        """生成现实化的船舶数据（改进版：支持马力需求）"""
        vessels = []
        
        duration_range, service_range, window_range, prep_time, _ = self._get_time_period_ranges(time_periods)
        
        # 按船型比例分配
        for ship_type, config in self.ship_type_config.items():
            count = int(vessel_num * config['ratio'])
            if ship_type == 'general':  
                count = vessel_num - len(vessels)
            
            for _ in range(count):
                vessel = {
                    'type': ship_type,
                    'size': random.randint(*config['size_range']),
                    'priority': round(random.uniform(*config['priority_range']), 1)
                }
                vessels.append(vessel)
        
        random.shuffle(vessels)
        
        # 生成到达时间
        vessel_etas = self._generate_realistic_etas(vessel_num, time_periods)
        
        # 泊位服务时间
        vessel_durations = []
        for vessel in vessels:
            if vessel['type'] == 'container':
                if vessel['size'] >= 4:
                    duration = random.randint(duration_range[1], duration_range[1])
                else:
                    duration = random.randint(duration_range[0], duration_range[1])
            elif vessel['type'] in ['tanker', 'bulk']:
                duration = random.randint(duration_range[0], duration_range[1])
            else:
                duration = random.randint(duration_range[0], max(duration_range[0], duration_range[1]-1))
            vessel_durations.append(duration)
        
        # 进港服务时间
        vessel_inbound_times = []
        for vessel in vessels:
            if vessel['size'] >= 4:
                inbound_time = random.randint(service_range[1], service_range[1])
            else:
                inbound_time = random.randint(service_range[0], service_range[1])
            vessel_inbound_times.append(inbound_time)
        
        # 离港服务时间
        vessel_outbound_times = []
        for vessel in vessels:
            if vessel['size'] >= 4:
                outbound_time = random.randint(service_range[1], service_range[1])
            else:
                outbound_time = random.randint(service_range[0], service_range[1])
            vessel_outbound_times.append(outbound_time)
        
        # 时间窗
        vessel_early_limits = []
        for vessel in vessels:
            if vessel['type'] == 'container':
                early_limit = random.randint(window_range[0], max(window_range[0], window_range[1]-1))
            else:
                early_limit = random.randint(window_range[0], window_range[1])
            vessel_early_limits.append(early_limit)
        
        vessel_late_limits = []
        for vessel in vessels:
            if vessel['type'] == 'container':
                late_limit = random.randint(window_range[0], max(window_range[0], window_range[1]-1))
            else:
                late_limit = random.randint(window_range[0], window_range[1])
            vessel_late_limits.append(late_limit)
        
        # 生成马力需求（基于船舶大小）
        vessel_horsepower_requirements = []
        for vessel in vessels:
            if vessel['size'] == 5:
                hp = random.randint(4500, 6000)
            elif vessel['size'] == 4:
                hp = random.randint(3000, 5000)
            elif vessel['size'] == 3:
                hp = random.randint(2000, 3500)
            elif vessel['size'] == 2:
                hp = random.randint(1200, 2200)
            else:  # size 1
                hp = random.randint(800, 1500)
            vessel_horsepower_requirements.append(hp)
        
        return (vessels, vessel_etas, vessel_durations, vessel_inbound_times, 
                vessel_outbound_times, vessel_early_limits, vessel_late_limits,
                vessel_horsepower_requirements)
    
    def _generate_realistic_etas(self, vessel_num, time_periods):
        """生成现实化的到达时间"""
        etas = []
        
        # 使用泊松分布生成到达时间
        lambda_param = vessel_num / (time_periods * 0.8)
        
        current_time = 0
        for _ in range(vessel_num):
            inter_arrival = max(1, int(np.random.exponential(1/lambda_param)))
            current_time += inter_arrival
            if current_time >= time_periods:
                current_time = random.randint(0, time_periods - 1)
            etas.append(current_time)
        
        etas.sort()
        
        # 添加随机扰动
        for i in range(len(etas)):
            perturbation = random.randint(-1, 1)
            etas[i] = max(0, min(time_periods - 1, etas[i] + perturbation))
        
        return etas
    
    def _generate_realistic_berths(self, berth_num, vessel_sizes):
        """生成现实化的泊位数据（容量等级系统）"""
        # 统计船舶大小分布
        size_counts = {}
        for size in vessel_sizes:
            size_counts[size] = size_counts.get(size, 0) + 1
        
        # 根据船舶需求分配泊位容量
        berth_capacities = []
        
        # 确保有足够的高容量泊位服务大型船舶
        high_cap_count = max(2, berth_num // 4)
        mid_cap_count = max(2, berth_num // 3)
        
        for _ in range(high_cap_count):
            berth_capacities.append(5)
        
        for _ in range(mid_cap_count):
            berth_capacities.append(random.choice([4, 4, 3]))
        
        remaining = berth_num - len(berth_capacities)
        for _ in range(remaining):
            berth_capacities.append(random.choice([2, 3, 3, 4]))
        
        random.shuffle(berth_capacities)
        
        return berth_capacities
    
    def _generate_realistic_tugboats(self, tugboat_num, vessel_horsepower_requirements):
        """生成现实化的拖船数据（马力系统）"""
        # 统计船舶马力需求分布
        max_hp_required = max(vessel_horsepower_requirements)
        avg_hp_required = np.mean(vessel_horsepower_requirements)
        
        # 生成拖船马力
        tugboat_horsepower = []
        
        # 确保有足够的高马力拖船组合来服务高需求船舶
        # 高马力拖船（可以单独或组合服务大船）
        high_hp_count = max(3, tugboat_num // 3)
        for _ in range(high_hp_count):
            hp = random.randint(3000, 5000)
            tugboat_horsepower.append(hp)
        
        # 中等马力拖船
        mid_hp_count = max(3, tugboat_num // 3)
        for _ in range(mid_hp_count):
            hp = random.randint(1500, 3500)
            tugboat_horsepower.append(hp)
        
        # 低马力拖船
        remaining = tugboat_num - len(tugboat_horsepower)
        for _ in range(remaining):
            hp = random.randint(800, 2000)
            tugboat_horsepower.append(hp)
        
        random.shuffle(tugboat_horsepower)
        
        # 生成拖船成本（与马力相关）
        tugboat_costs = []
        for hp in tugboat_horsepower:
            if hp >= 4000:
                cost = round(random.uniform(80, 130), 1)
            elif hp >= 3000:
                cost = round(random.uniform(60, 100), 1)
            elif hp >= 2000:
                cost = round(random.uniform(40, 75), 1)
            elif hp >= 1000:
                cost = round(random.uniform(25, 50), 1)
            else:
                cost = round(random.uniform(15, 35), 1)
            tugboat_costs.append(cost)
        
        return tugboat_horsepower, tugboat_costs
    
    def _get_time_unit_info(self, time_periods):
        """获取时间单位信息"""
        if time_periods == 12:
            return "2 hours", 2
        elif time_periods == 24:
            return "1 hour", 1
        elif time_periods == 48:
            return "30 minutes", 0.5
        else:
            return "1 hour (default)", 1
    
    def _print_horsepower_matching_analysis(self, vessel_horsepower_requirements, tugboat_horsepower, max_tugboats):
        """打印马力匹配分析"""
        print(f"\n马力匹配性分析:")
        
        # 船舶马力需求分布
        hp_ranges = {
            '800-1500': 0,
            '1500-2500': 0,
            '2500-3500': 0,
            '3500-4500': 0,
            '4500-6000': 0
        }
        
        for hp in vessel_horsepower_requirements:
            if hp < 1500:
                hp_ranges['800-1500'] += 1
            elif hp < 2500:
                hp_ranges['1500-2500'] += 1
            elif hp < 3500:
                hp_ranges['2500-3500'] += 1
            elif hp < 4500:
                hp_ranges['3500-4500'] += 1
            else:
                hp_ranges['4500-6000'] += 1
        
        print(f"  船舶马力需求分布:")
        for range_name, count in hp_ranges.items():
            print(f"    {range_name} HP: {count}艘")
        
        # 拖船马力分布
        tug_hp_ranges = {
            '800-1500': 0,
            '1500-2500': 0,
            '2500-3500': 0,
            '3500-4500': 0,
            '4500-5000': 0
        }
        
        for hp in tugboat_horsepower:
            if hp < 1500:
                tug_hp_ranges['800-1500'] += 1
            elif hp < 2500:
                tug_hp_ranges['1500-2500'] += 1
            elif hp < 3500:
                tug_hp_ranges['2500-3500'] += 1
            elif hp < 4500:
                tug_hp_ranges['3500-4500'] += 1
            else:
                tug_hp_ranges['4500-5000'] += 1
        
        print(f"  拖船马力分布:")
        for range_name, count in tug_hp_ranges.items():
            print(f"    {range_name} HP: {count}艘")
        
        print(f"\n  最大马力需求: {max(vessel_horsepower_requirements)} HP")
        print(f"  最大拖船马力: {max(tugboat_horsepower)} HP")
        print(f"  最多{max_tugboats}艘拖船组合可提供: {sum(sorted(tugboat_horsepower, reverse=True)[:max_tugboats])} HP")
    
    def _print_balanced_cost_analysis(self, penalty_parameter, objective_weights, 
                                     vessel_waiting_costs, vessel_jit_costs, tugboat_costs,
                                     vessels, vessel_durations):
        """打印平衡成本分析"""
        print(f"\n平衡成本参数:")
        print(f"  惩罚参数 M = {penalty_parameter:.1f}")
        print(f"  目标权重 λ = [{objective_weights[0]:.3f}, {objective_weights[1]:.3f}, {objective_weights[2]:.3f}, {objective_weights[3]:.3f}]")
        print(f"  船舶等待成本范围: [{min(vessel_waiting_costs):.1f}, {max(vessel_waiting_costs):.1f}]")
        print(f"  JIT偏离成本范围: [{min(vessel_jit_costs):.1f}, {max(vessel_jit_costs):.1f}]")
        print(f"  拖船使用成本范围: [{min(tugboat_costs):.1f}, {max(tugboat_costs):.1f}]")
    
    def _print_realistic_analysis(self, vessel_sizes, berth_capacities, tugboat_horsepower,
                                 vessel_num, berth_num, tugboat_num, time_periods,
                                 vessel_durations, vessel_inbound_times, vessel_outbound_times):
        """打印现实性分析"""
        print(f"\n现实性分析:")
        print(f"  船舶数量: {vessel_num}, 泊位数量: {berth_num}, 拖船数量: {tugboat_num}")
        print(f"  计划周期: T={time_periods}")
        print(f"  平均泊位服务时间: {np.mean(vessel_durations):.2f} 周期")
        print(f"  平均进港时间: {np.mean(vessel_inbound_times):.2f} 周期")
        print(f"  平均离港时间: {np.mean(vessel_outbound_times):.2f} 周期")
        
        avg_total_time = np.mean(vessel_durations) + np.mean(vessel_inbound_times) + np.mean(vessel_outbound_times)
        theoretical_capacity = time_periods / avg_total_time
        utilization = vessel_num / (berth_num * theoretical_capacity)
        
        print(f"  理论单泊位容量: {theoretical_capacity:.2f} 艘船/周期")
        print(f"  泊位利用率估计: {utilization*100:.1f}%")
    
    def generate_balanced_case(self, vessel_num, berth_num, tugboat_num, time_periods, 
                              case_name, output_dir="balanced_cases_horsepower"):
        """生成平衡的多目标测试用例（马力约束系统）"""
        os.makedirs(output_dir, exist_ok=True)
        
        # 生成船舶数据
        (vessels, vessel_etas, vessel_durations, vessel_inbound_times,
         vessel_outbound_times, vessel_early_limits, vessel_late_limits,
         vessel_horsepower_requirements) = \
            self._generate_realistic_vessels(vessel_num, time_periods)
        
        vessel_sizes = [v['size'] for v in vessels]
        vessel_priorities = [v['priority'] for v in vessels]
        
        # 生成泊位数据
        berth_capacities = self._generate_realistic_berths(berth_num, vessel_sizes)
        
        # 生成拖船数据（马力系统）
        tugboat_horsepower, tugboat_costs = self._generate_realistic_tugboats(tugboat_num, vessel_horsepower_requirements)
        
        # 计算平衡的成本参数
        penalty_parameter, objective_weights, vessel_waiting_costs, vessel_jit_costs = \
            self._calculate_balanced_costs(vessels, vessel_durations, tugboat_costs, time_periods)
        
        # 系统参数
        _, _, _, prep_time, tolerance = self._get_time_period_ranges(time_periods)
        inbound_preparation_time = prep_time
        outbound_preparation_time = prep_time
        
        # 最多允许的拖船组合数
        if time_periods == 12:
            max_tugboats_per_service = 2
        elif time_periods == 24:
            max_tugboats_per_service = 2
        else:  # 48
            max_tugboats_per_service = 2
        
        time_constraint_tolerance = tolerance
        
        # 写入文件
        filename = os.path.join(output_dir, f"{case_name}.txt")
        time_unit, hours_per_period = self._get_time_unit_info(time_periods)
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("# Balanced Multi-Objective Port Scheduling Problem Instance with Horsepower Constraints\n")
            f.write("# Mathematical model with multi-tugboat combinations and horsepower requirements\n")
            f.write(f"# Time configuration: T={time_periods}, each period = {time_unit}\n")
            f.write("# Format: parameter_name = value or [array values]\n\n")
            
            # 基本维度
            f.write("# Basic dimensions\n")
            f.write(f"vessel_num = {vessel_num}\n")
            f.write(f"berth_num = {berth_num}\n")
            f.write(f"tugboat_num = {tugboat_num}\n")
            f.write(f"time_periods = {time_periods}\n\n")
            
            # 船舶数据
            f.write(f"# Vessel data ({vessel_num} vessels) - All times in INTEGER periods\n")
            f.write(f"vessel_sizes = {vessel_sizes}\n")
            f.write(f"vessel_etas = {vessel_etas}\n")
            f.write(f"vessel_durations = {vessel_durations}\n")
            f.write(f"vessel_inbound_service_times = {vessel_inbound_times}\n")
            f.write(f"vessel_outbound_service_times = {vessel_outbound_times}\n")
            f.write(f"vessel_priority_weights = {vessel_priorities}\n")
            f.write(f"vessel_waiting_costs = {vessel_waiting_costs}\n")
            f.write(f"vessel_jit_costs = {vessel_jit_costs}\n")
            f.write(f"vessel_horsepower_requirements = {vessel_horsepower_requirements}\n")
            f.write(f"vessel_early_limits = {vessel_early_limits}\n")
            f.write(f"vessel_late_limits = {vessel_late_limits}\n\n")
            
            # 泊位数据
            f.write(f"# Berth data ({berth_num} berths)\n")
            f.write(f"berth_capacities = {berth_capacities}\n\n")
            
            # 拖船数据 - 马力系统
            f.write(f"# Tugboat data ({tugboat_num} tugboats) - Horsepower-based system\n")
            f.write(f"tugboat_horsepower = {tugboat_horsepower}\n")
            f.write(f"tugboat_costs = {tugboat_costs}\n\n")
            
            # 系统参数
            f.write("# System parameters for multi-tugboat combinations\n")
            f.write(f"inbound_preparation_time = {inbound_preparation_time}\n")
            f.write(f"outbound_preparation_time = {outbound_preparation_time}\n")
            f.write(f"max_tugboats_per_service = {max_tugboats_per_service}\n")
            f.write(f"time_constraint_tolerance = {time_constraint_tolerance}\n")
            f.write(f"penalty_parameter = {penalty_parameter}\n")
            f.write(f"objective_weights = {objective_weights}\n\n")
            
            # 数学模型参数
            f.write("# Mathematical model parameters (horsepower-based)\n")
            f.write(f"M = {penalty_parameter}\n")
            f.write(f"lambda_1 = {objective_weights[0]}\n")
            f.write(f"lambda_2 = {objective_weights[1]}\n")
            f.write(f"lambda_3 = {objective_weights[2]}\n")
            f.write(f"lambda_4 = {objective_weights[3]}\n")
            f.write(f"epsilon_time = {time_constraint_tolerance}\n")
            f.write(f"rho_in = {inbound_preparation_time}\n")
            f.write(f"rho_out = {outbound_preparation_time}\n")
            f.write(f"H_max = {max_tugboats_per_service}\n")
            f.write(f"tau_in = {vessel_inbound_times}\n")
            f.write(f"tau_out = {vessel_outbound_times}\n")
            f.write(f"alpha = {vessel_priorities}\n")
            f.write(f"beta = {vessel_waiting_costs}\n")
            f.write(f"gamma = {vessel_jit_costs}\n")
            f.write(f"P_k = {tugboat_horsepower}\n")
            f.write(f"P_req = {vessel_horsepower_requirements}\n")
            f.write(f"c_k = {tugboat_costs}\n")
            f.write(f"Delta_early = {vessel_early_limits}\n")
            f.write(f"Delta_late = {vessel_late_limits}\n")
        
        # 打印马力匹配分析
        self._print_horsepower_matching_analysis(vessel_horsepower_requirements, tugboat_horsepower, max_tugboats_per_service)
        
        # 打印平衡成本分析
        self._print_balanced_cost_analysis(penalty_parameter, objective_weights, 
                                         vessel_waiting_costs, vessel_jit_costs, tugboat_costs,
                                         vessels, vessel_durations)
        
        # 打印基础分析
        self._print_realistic_analysis(vessel_sizes, berth_capacities, tugboat_horsepower,
                                     vessel_num, berth_num, tugboat_num, time_periods,
                                     vessel_durations, vessel_inbound_times, vessel_outbound_times)
        
        print(f"\n生成平衡多目标用例: {filename}")
        return filename


# 生成32个随机实例
if __name__ == "__main__":
    generator = ImprovedRealisticPortDataGenerator(seed=42)
    
    print("=" * 80)
    print("=== 生成32个随机规模测试实例 (马力约束系统) ===")
    print("=" * 80)
    
    # 生成32个随机实例
    for i in range(1, 33):
        # 随机船舶数量: 100-500
        vessel_num = random.randint(100, 500)
        
        # 根据船舶数量随机生成其他参数
        # 泊位数量: 约为船舶数的 8%-15%
        berth_num = random.randint(int(vessel_num * 0.08), int(vessel_num * 0.15))
        berth_num = max(10, berth_num)  # 至少10个泊位
        
        # 拖船数量: 约为船舶数的 10%-20%
        tugboat_num = random.randint(int(vessel_num * 0.10), int(vessel_num * 0.20))
        tugboat_num = max(15, tugboat_num)  # 至少15艘拖船
        
        # 随机选择时间周期: 12, 24, 或 48
        time_periods = random.choice([12, 24, 48])
        
        # 生成实例名称 - 保持原有命名风格
        case_name = f"{vessel_num}_{berth_num}_{tugboat_num}_T{time_periods}"
        
        print(f"\n{'='*80}")
        print(f"正在生成实例 {i}/32: {case_name}")
        print(f"  参数: {vessel_num}艘船, {berth_num}个泊位, {tugboat_num}艘拖船, T={time_periods}")
        print(f"{'='*80}")
        
        generator.generate_balanced_case(
            vessel_num=vessel_num,
            berth_num=berth_num,
            tugboat_num=tugboat_num,
            time_periods=time_periods,
            case_name=case_name,
            output_dir="horsepower_32_instances"
        )
    
    print(f"\n{'='*80}")
    print("=== 所有32个测试实例生成完毕 (马力约束系统) ===")
    print(f"{'='*80}")
    print("所有实例保存在 ./horsepower_32_instances/ 目录下")
    print("\n实例特点:")
    print("- 总数量: 32个实例")
    print("- 船舶数量: 100-500艘 (随机)")
    print("- 泊位数量: 约为船舶数的 8%-15% (随机)")
    print("- 拖船数量: 约为船舶数的 10%-20% (随机)")
    print("- 时间周期: 12, 24, 或 48 (随机)")
    print("\n马力约束系统特点:")
    print("- 船舶有具体的马力需求(vessel_horsepower_requirements: 800-6000 HP)")
    print("- 拖船有具体的马力供给(tugboat_horsepower: 800-5000 HP)")
    print("- 支持多拖船组合服务(max_tugboats_per_service: 最多2艘)")
    print("- 马力约束确保组合拖船总马力 ≥ 船舶需求")
    print("- 智能匹配高马力需求船舶与高马力拖船")
    print("\n文件命名格式:")
    print("  [船数]_[泊位数]_[拖船数]_T[时间周期].txt")
    print("  示例: 350_42_65_T24.txt")

=== 生成32个随机规模测试实例 (马力约束系统) ===

正在生成实例 1/32: 427_37_43_T48
  参数: 427艘船, 37个泊位, 43艘拖船, T=48
成本估算 - Z2典型值: 744.1, Z3典型值: 92.7, Z4典型值: 251.7
总量级估算 - Z1: 134592, Z2: 317725, Z3: 39568, Z4: 107493

马力匹配性分析:
  船舶马力需求分布:
    800-1500 HP: 32艘
    1500-2500 HP: 83艘
    2500-3500 HP: 125艘
    3500-4500 HP: 76艘
    4500-6000 HP: 111艘
  拖船马力分布:
    800-1500 HP: 6艘
    1500-2500 HP: 18艘
    2500-3500 HP: 8艘
    3500-4500 HP: 6艘
    4500-5000 HP: 5艘

  最大马力需求: 5989 HP
  最大拖船马力: 4986 HP
  最多2艘拖船组合可提供: 9766 HP

平衡成本参数:
  惩罚参数 M = 2721.2
  目标权重 λ = [0.215, 0.192, 0.324, 0.112]
  船舶等待成本范围: [15.7, 70.0]
  JIT偏离成本范围: [5.5, 32.0]
  拖船使用成本范围: [17.2, 129.0]

现实性分析:
  船舶数量: 427, 泊位数量: 37, 拖船数量: 43
  计划周期: T=48
  平均泊位服务时间: 4.42 周期
  平均进港时间: 3.52 周期
  平均离港时间: 3.48 周期
  理论单泊位容量: 4.20 艘船/周期
  泊位利用率估计: 274.8%

生成平衡多目标用例: horsepower_32_instances\427_37_43_T48.txt

正在生成实例 2/32: 266_21_31_T24
  参数: 266艘船, 21个泊位, 31艘拖船, T=24
成本估算 - Z2典型值: 487.4, Z3典型值: 92.0, Z4典型值: 237.9
总量级估算 - Z1: 62528, Z2: 129647, Z3: 24480, Z4: 6